**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -p 8786:8786 -p 8787:8787 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Esta nota utiliza métodos vistos en [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb)

# El módulo de `multiprocessing`

Documentación en: [multiprocessing](https://docs.python.org/3.1/library/multiprocessing.html).

La implementación más utilizada de Python, [CPython](https://en.wikipedia.org/wiki/CPython) no utiliza múltiples cores por default. Se recomienda leer la discusión de la liga anterior en el apartado *Design* sobre el [Global INterpreter Lock: GIL](https://en.wikipedia.org/wiki/Global_interpreter_lock) y el por qué CPython no soporta ejecución *multithreaded* o *multiprocesses*.

El módulo `multiprocessing` nos permite realizar procesamientos basados en procesos o threads para compartir trabajo y datos. Se recomienda usar este módulo para el *shared memory programming* (ver [2.2.Sistemas_de_memoria_compartida](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.2.Sistemas_de_memoria_compartida.ipynb)) y para trabajos que son demandantes de CPU. Para paralelizar trabajos demandantes en I/O no se recomienda su uso.

**Otro módulo en Python para procesamiento utilizando los cores de tu máquina es [concurrent.features](https://docs.python.org/3/library/concurrent.futures.html) que provee el comportamiento principal de `multiprocessing`**. Ver [liga](https://stackoverflow.com/questions/38311431/concurrent-futures-processpoolexecutor-vs-multiprocessing-pool-pool?noredirect=1&lq=1) y [liga](https://stackoverflow.com/questions/20776189/concurrent-futures-vs-multiprocessing-in-python-3) para más sobre `concurrent.futures` y `concurrent.futures` vs `multiprocessing`.

## Nota sobre el GIL y `multiprocessing`

Aunque en Python los threads son nativos del sistema operativo (esto es, no se simula, son realmente threads del sistema operativo), están limitados por el *global interpreter lock, GIL*, de modo que un sólo thread interactúe con un objeto Python en un único tiempo.

Al usar el módulo `multiprocessing` ejecutamos en paralelo un número de **interpretadores Python** (CPython), cada uno con su propio espacio de memoria privada y su propio GIL que se ejecutan en un instante (y con un thread). 

**Comentario:** en `multiprocessing` se utilizan subprocesos en lugar de threads.

# Ejemplos

## 1) Hello world!

In [1]:
import multiprocessing
from multiprocessing import Process
from multiprocessing import Pool

In [2]:
def f():
    print('hello world!')
    
if __name__=='__main__':
    p1 = Process(target=f)
    p2 = Process(target=f)
    p1.start()
    p2.start()
    p1.join()
    p2.join()

hello world!
hello world!


In [3]:
def f(s):
    print(s)
    
if __name__=='__main__':
    p1 = Process(target=f, args=('hola',))
    p2 = Process(target=f, args=('mundo',)) 
    p1.start()
    p2.start()
    p1.join()
    p2.join()

hola
mundo


**Pool of workers, ver [Using a pool of workers](https://docs.python.org/3/library/multiprocessing.html#using-a-pool-of-workers)**

En *multiprocessing* tenemos la función `cpu_count` para determinar el número de cores que el sistema operativo puede usar. Este número es la cantidad física o simulada (hyperthreading) de cores.

In [4]:
multiprocessing.cpu_count()

2

In [5]:
def f(dummy):
    return 'hello world!'
    
if __name__ == '__main__':
    pool = Pool(multiprocessing.cpu_count())
    pool_map = pool.map(f,range(multiprocessing.cpu_count()))
    print(pool_map)
    pool.close()    
    pool.join()

['hello world!', 'hello world!']


In [6]:
def f(dummy):
    return 'hello world!'
    
if __name__ == '__main__':
    num_processes=2
    pool = Pool(num_processes)
    pool_map = pool.map(f,range(num_processes))
    print(pool_map)
    pool.close()    
    pool.join()

['hello world!', 'hello world!']


In [7]:
def f(dummy):
    return 'hello world!'
    
if __name__ == '__main__':
    num_processes=2
    with Pool(processes=num_processes) as pool:
        pool_map = pool.map(f,range(num_processes))
        print(pool_map)

['hello world!', 'hello world!']


In [8]:
def f(s):
    return s
    
if __name__ == '__main__':
    num_processes=2
    with Pool(processes=num_processes) as pool:
        pool_map = pool.starmap(f,[('hola',),('mundo',)])
        print(pool_map)

['hola', 'mundo']


## 2) Regla compuesta del rectángulo

In [9]:
import math
import time
from scipy.integrate import quad

In [10]:
f=lambda x: math.exp(-x**2)
a=0
b=1

In [11]:
def Rcf(f, a, b, n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    sum_res=0
    for i in range(0,n):
        x=a+(i+1/2.0)*h_hat
        sum_res+=f(x)
    return h_hat*sum_res

In [12]:
n=10**6
start_time = time.time()
aprox=Rcf(f,a,b,n)
end_time = time.time()

In [13]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.3412818908691406 segundos


In [14]:
obj, err = quad(f, 0, 1)

In [15]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [16]:
err_relativo(aprox,obj)

6.71939731300312e-14

In [17]:
p=2 #número de procesos
ns_p=int(n/p) #número de subintervalos por proceso
              #se asume que n es divisible por p
              #si no se cumple esto, se puede usar 
              #ns_p=int(n/p)
              #y definir 
              #n=p*ns_p

In [18]:
print("número de subintervalos:",n)

número de subintervalos: 1000000


In [19]:
def Rcf_parallel(mi_id):
    begin=mi_id*ns_p
    end=begin+ns_p
    h_hat=(b-a)/n
    sum_res=0
    for i in range(begin,end):
        x=a+(i+1/2.0)*h_hat
        sum_res+=f(x)
    return h_hat*sum_res
if __name__ == '__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        pool_map = pool.map(Rcf_parallel,range(p))
        aprox=sum(pool_map)
    end_time=time.time()

In [20]:
secs = end_time-start_time
print("Rcf_parallel tomó",secs,"segundos" )

Rcf_parallel tomó 0.32051730155944824 segundos


In [21]:
err_relativo(aprox,obj)

5.842307840730588e-14

In [22]:
def f(x):
    return math.exp(-x**2)

Ejemplo para pasar múltiples parámetros a una función vía un [generator](https://wiki.python.org/moin/Generators).

In [23]:
def Rcf_parallel2(t):
    fun,a,b,mi_id = t
    begin=mi_id*ns_p
    end=begin+ns_p
    h_hat=(b-a)/n
    sum_res=0
    for i in range(begin,end):
        x=a+(i+1/2.0)*h_hat
        sum_res+=fun(x)
    return h_hat*sum_res
if __name__ == '__main__':
    it=((f,0,1,k) for k in range(p))
    start_time=time.time()
    with Pool(processes=p) as pool:
        pool_map = pool.map(Rcf_parallel2,it)
        aprox=sum(pool_map)
    end_time=time.time()

In [24]:
secs = end_time-start_time
print("Rcf_parallel tomó",secs,"segundos" )

Rcf_parallel tomó 0.32392048835754395 segundos


In [25]:
err_relativo(aprox,obj)

5.842307840730588e-14

**Referencias**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.
